In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from dolfin import *

# Sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[0]) < DOLFIN_EPS) or (abs(x[0] - 1.0) < DOLFIN_EPS) 
        #return abs(x[0] - 1.0) < DOLFIN_EPS and on_boundary

# Create mesh and define function space
mesh = UnitSquareMesh(100, 100)
File("mesh.pvd") << mesh

V = FunctionSpace(mesh, "CG", 1)

# Define boundary condition
g = Constant(313.0)

# Define variational problem
u = Function(V)
v = TestFunction(V)

u_0 = 4000
u = project(u_0,V)

#f = Expression("x[0]*sin(x[1])", degree=2)
sigma = 1.387 * pow(10,-23)
epsilon = 0.73
T_0 = 313.0
h_conv = 60.0
kappa_0 = 2.03


g2 = Expression("((abs(x[1] - 1.0) < 1e-4) && (x[0]>0.49) && (x[0]<0.51)) ? 600000.0:0.0", degree=1)
F = inner(grad(u), grad(v))*dx - kappa_0*g2*v*ds + kappa_0*epsilon*sigma*u**4*v*ds - kappa_0*epsilon*sigma*T_0**4*v*ds + kappa_0*h_conv*(u)*v*ds - kappa_0*h_conv*(T_0)*v*ds

# Compute solution
solve(F == 0, u, solver_parameters={"newton_solver":{"relative_tolerance":1e-3},
                                   "newton_solver":{"maximum_iterations":1000}})

# Plot solution and solution gradient
plt.figure()
p = plot(u)

plt.colorbar(p)
plt.interactive(True)

plt.figure()
plot(grad(u), title="Solution gradient")

plt.show()

# Save solution in VTK format
file = File("nonlinear_poisson.pvd")
file << u